<h1><center>D210: Representation and Reporting</center></h1>
<br>
<center></center>
<br>
<center>Department of Information Technology, Western Governor's University</center>
<br>
<center>Dr. Kesselly Kamara</center>
<br>
<center>April 11, 2024</center>
<br>
<br>
<br>
<br>

## A1. Datasets
For this project, I will be using the 2017-2018 NHANES dataset collected by the CDC as my external dataset. This dataset comes fragmented in several SAS files called XPT files. Below, I will use Python to clean up these files so they are more human readable, have no missing data, and are comparable to the WGU data in terms of how they handle people older than 80 and people younger than 18 (WGU does not include minors while the CDC does, and the CDC top-codes all people older than 80 while WGU doesn't.) Then, I will merge the various CDC files together into one, and ultimately merge the CDC data to the WGU data so I don't have to deal with making relationships in Tableau. In order to clean up the CDC files, I have used the multiple data dictionaries provided by them for column header and value translation.

* The CDC NHANES Dataset can be found here: https://wwwn.cdc.gov/nchs/nhanes/continuousnhanes/default.aspx?BeginYear=2017
* The CDC's Demographic data dictionary can be found here: https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/DEMO_J.htm
* The CDC's Medical Conditions data dictionary can be found here: https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/MCQ_J.htm
* The CDC's Diabetes data dictionary can be found here: https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/DIQ_J.htm
* The CDC's Blood Pressure and Cholesterol data dictionary can be found here: https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/BPQ_J.htm

In addition to cleaning the CDC data, the WGU data will also be cleaned because, as usual, it having been stored in a CSV file caused field datatypes to be incorrect, removed leading zeros from the zip codes, and more. Thus, I will reuse some code from D206 to clean up this file a bit before merging it with the CDC data.  The CDC data also doesn't contain every variable WGU's data does and vice versa, so I will be removing any columns that the two data sources don't both share. The CDC data contains a plethora of information I'll not be analyzing because there's simply nothing in the WGU data that's comparable.

Though there's no requirement I provide my cleaning code, I have done so anyway, in case it is of interest. I will use the code below to export my final datasets prior to merging the WGU and CDC data together, which should fulfill the requirement to provide such data. Just in case, I will also provide my final, merged dataset as well.

In [1]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np

#Import NHANES data, which is an XPT (SAS) file. Check dataframe post-load
demo_df = pd.read_sas('C:/Users/essay/Documents/D210 PA Dataset/demographics.XPT', index='SEQN')
pd.options.display.float_format = '{:.0f}'.format
demo_df.head(10)

,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,DMQMILIZ,...,DMDHREDZ,DMDHRMAZ,DMDHSEDZ,WTINT2YR,WTMEC2YR,SDMVPSU,SDMVSTRA,INDHHIN2,INDFMIN2,INDFMPIR
SEQN,,,,,,,,,,,,,,,,,,,,,
93703,10,2,2,2,NaN,5,6,2,27,NaN,...,3,1,3,9246,8540,2,145,15,15,5
93704,10,2,1,2,NaN,3,3,1,33,NaN,...,3,1,2,37339,42567,1,143,15,15,5
93705,10,2,2,66,NaN,4,4,2,NaN,2,...,1,2,NaN,8615,8338,2,145,3,3,1
93706,10,2,1,18,NaN,5,6,2,222,2,...,3,1,2,8549,8723,2,134,NaN,NaN,NaN
93707,10,2,1,13,NaN,5,7,2,158,NaN,...,2,1,3,6769,7065,1,138,10,10,2
93708,10,2,2,66,NaN,5,6,2,NaN,2,...,1,1,1,13329,14372,2,138,6,6,2
93709,10,2,2,75,NaN,4,4,1,NaN,2,...,2,2,NaN,12043,12278,1,136,2,2,0
93710,10,2,2,0,11,3,3,2,13,NaN,...,3,1,3,16418,16848,1,134,15,15,5
93711,10,2,1,56,NaN,5,6,2,NaN,2,...,3,1,3,11178,12391,2,134,15,15,5


In [2]:
demo_df.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 9254 entries, 93703.0 to 102956.0
Data columns (total 45 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SDDSRVYR  9254 non-null   float64
 1   RIDSTATR  9254 non-null   float64
 2   RIAGENDR  9254 non-null   float64
 3   RIDAGEYR  9254 non-null   float64
 4   RIDAGEMN  597 non-null    float64
 5   RIDRETH1  9254 non-null   float64
 6   RIDRETH3  9254 non-null   float64
 7   RIDEXMON  8704 non-null   float64
 8   RIDEXAGM  3433 non-null   float64
 9   DMQMILIZ  6004 non-null   float64
 10  DMQADFC   561 non-null    float64
 11  DMDBORN4  9254 non-null   float64
 12  DMDCITZN  9251 non-null   float64
 13  DMDYRSUS  1948 non-null   float64
 14  DMDEDUC3  2306 non-null   float64
 15  DMDEDUC2  5569 non-null   float64
 16  DMDMARTL  5569 non-null   float64
 17  RIDEXPRG  1110 non-null   float64
 18  SIALANG   9254 non-null   float64
 19  SIAPROXY  9254 non-null   float64
 20  SIAINTRP  9254 non

In [3]:
# WGU dataset doesn't separate # children into age buckets. So here, I will combine these columns.
demo_df['Children'] = demo_df["DMDHHSZA"] + demo_df["DMDHHSZB"]

#Check df to see if changes took
pd.set_option('display.max_columns', None)
demo_df.head(2)

,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,DMQMILIZ,DMQADFC,DMDBORN4,DMDCITZN,DMDYRSUS,DMDEDUC3,DMDEDUC2,DMDMARTL,RIDEXPRG,SIALANG,SIAPROXY,SIAINTRP,FIALANG,FIAPROXY,FIAINTRP,MIALANG,MIAPROXY,MIAINTRP,AIALANGA,DMDHHSIZ,DMDFMSIZ,DMDHHSZA,DMDHHSZB,DMDHHSZE,DMDHRGND,DMDHRAGZ,DMDHREDZ,DMDHRMAZ,DMDHSEDZ,WTINT2YR,WTMEC2YR,SDMVPSU,SDMVSTRA,INDHHIN2,INDFMIN2,INDFMPIR,Children
SEQN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
93703,10,2,2,2,NaN,5,6,2,27,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,1,1,2,1,2,2,NaN,NaN,NaN,NaN,5,5,3,0,0,1,2,3,1,3,9246,8540,2,145,15,15,5,3
93704,10,2,1,2,NaN,3,3,1,33,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,1,1,2,1,2,2,NaN,NaN,NaN,NaN,4,4,2,0,0,1,2,3,1,2,37339,42567,1,143,15,15,5,2


In [4]:
# Make the column that holds gender human-readable and more akin to WGU dataset (Male and Female)
gender_mapping = {2.0: 'Female', 1.0: 'Male'}
demo_df['RIAGENDR'] = demo_df['RIAGENDR'].map(gender_mapping)

#Check if changes took
demo_df.head(2)

,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,DMQMILIZ,DMQADFC,DMDBORN4,DMDCITZN,DMDYRSUS,DMDEDUC3,DMDEDUC2,DMDMARTL,RIDEXPRG,SIALANG,SIAPROXY,SIAINTRP,FIALANG,FIAPROXY,FIAINTRP,MIALANG,MIAPROXY,MIAINTRP,AIALANGA,DMDHHSIZ,DMDFMSIZ,DMDHHSZA,DMDHHSZB,DMDHHSZE,DMDHRGND,DMDHRAGZ,DMDHREDZ,DMDHRMAZ,DMDHSEDZ,WTINT2YR,WTMEC2YR,SDMVPSU,SDMVSTRA,INDHHIN2,INDFMIN2,INDFMPIR,Children
SEQN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
93703,10,2,Female,2,NaN,5,6,2,27,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,1,1,2,1,2,2,NaN,NaN,NaN,NaN,5,5,3,0,0,1,2,3,1,3,9246,8540,2,145,15,15,5,3
93704,10,2,Male,2,NaN,3,3,1,33,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,1,1,2,1,2,2,NaN,NaN,NaN,NaN,4,4,2,0,0,1,2,3,1,2,37339,42567,1,143,15,15,5,2


In [5]:
# Rename columns to make them human-readable and more akin to those in WGU dataset.
demo_df.rename(columns = {'RIAGENDR': "Gender", 'RIDAGEYR' : 'Age'}, inplace = True)

#Check df again
demo_df.head(10)

,SDDSRVYR,RIDSTATR,Gender,Age,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,DMQMILIZ,DMQADFC,DMDBORN4,DMDCITZN,DMDYRSUS,DMDEDUC3,DMDEDUC2,DMDMARTL,RIDEXPRG,SIALANG,SIAPROXY,SIAINTRP,FIALANG,FIAPROXY,FIAINTRP,MIALANG,MIAPROXY,MIAINTRP,AIALANGA,DMDHHSIZ,DMDFMSIZ,DMDHHSZA,DMDHHSZB,DMDHHSZE,DMDHRGND,DMDHRAGZ,DMDHREDZ,DMDHRMAZ,DMDHSEDZ,WTINT2YR,WTMEC2YR,SDMVPSU,SDMVSTRA,INDHHIN2,INDFMIN2,INDFMPIR,Children
SEQN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
93703,10,2,Female,2,NaN,5,6,2,27,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,1,1,2,1,2,2,NaN,NaN,NaN,NaN,5,5,3,0,0,1,2,3,1,3,9246,8540,2,145,15,15,5,3
93704,10,2,Male,2,NaN,3,3,1,33,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,1,1,2,1,2,2,NaN,NaN,NaN,NaN,4,4,2,0,0,1,2,3,1,2,37339,42567,1,143,15,15,5,2
93705,10,2,Female,66,NaN,4,4,2,NaN,2,NaN,1,1,NaN,NaN,2,3,NaN,1,2,2,1,2,2,1,2,2,1,1,1,0,0,1,2,4,1,2,NaN,8615,8338,2,145,3,3,1,0
93706,10,2,Male,18,NaN,5,6,2,222,2,NaN,1,1,NaN,15,NaN,NaN,NaN,1,2,2,NaN,NaN,NaN,1,2,2,1,5,5,0,0,1,1,4,3,1,2,8549,8723,2,134,NaN,NaN,NaN,0
93707,10,2,Male,13,NaN,5,7,2,158,NaN,NaN,1,1,NaN,6,NaN,NaN,NaN,1,1,2,1,2,2,1,2,2,1,7,7,0,3,0,1,3,2,1,3,6769,7065,1,138,10,10,2,3
93708,10,2,Female,66,NaN,5,6,2,NaN,2,NaN,2,1,7,NaN,1,1,NaN,1,2,1,1,2,2,1,2,1,3,2,2,0,0,2,1,4,1,1,1,13329,14372,2,138,6,6,2,0
93709,10,2,Female,75,NaN,4,4,1,NaN,2,NaN,1,1,NaN,NaN,4,2,NaN,1,2,2,1,2,2,NaN,NaN,NaN,NaN,1,1,0,0,1,2,4,2,2,NaN,12043,12278,1,136,2,2,0,0
93710,10,2,Female,0,11,3,3,2,13,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,1,1,2,1,2,2,NaN,NaN,NaN,NaN,3,3,1,0,0,1,2,3,1,3,16418,16848,1,134,15,15,5,1
93711,10,2,Male,56,NaN,5,6,2,NaN,2,NaN,2,1,6,NaN,5,1,NaN,1,2,2,1,2,2,1,2,2,1,3,3,0,0,0,1,3,3,1,3,11178,12391,2,134,15,15,5,0


In [6]:
# Reduce demo_df down to columns I'm interested in.
demo_df = demo_df[['Age', 'Gender', 'Children']]

# Check df
demo_df.head(5)

,Age,Gender,Children
SEQN,,,
93703,2,Female,3
93704,2,Male,2
93705,66,Female,0
93706,18,Male,0
93707,13,Male,3


In [7]:
# Load questionnaire/cardio health df, which is an XPT (SAS) file. Check dataframe post-load
cardio_df = pd.read_sas('C:/Users/essay/Documents/D210 PA Dataset/Cardio.XPT', index='SEQN')
cardio_df.head(10)

,BPQ020,BPQ030,BPD035,BPQ040A,BPQ050A,BPQ080,BPQ060,BPQ070,BPQ090D,BPQ100D
SEQN,,,,,,,,,,
93705,1,1,50,1,1,2,1,2,2,NaN
93706,2,NaN,NaN,NaN,NaN,2,1,2,2,NaN
93708,1,1,50,1,1,1,NaN,1,1,1
93709,1,1,71,1,1,2,1,2,2,NaN
93711,2,NaN,NaN,NaN,NaN,1,NaN,1,1,2
93712,2,NaN,NaN,NaN,NaN,2,2,NaN,NaN,NaN
93713,2,NaN,NaN,NaN,NaN,2,9,NaN,NaN,NaN
93714,2,NaN,NaN,NaN,NaN,1,NaN,2,1,2
93715,2,NaN,NaN,NaN,NaN,1,NaN,1,1,1


In [8]:
# Rename columns to make them human-readable and more akin to those in WGU dataset.
cardio_df.rename(columns = {'BPQ080': 'Hyperlipidemia', 'BPQ020' : 'HighBlood'}, inplace = True)

# Check df.
cardio_df.head(5)

,HighBlood,BPQ030,BPD035,BPQ040A,BPQ050A,Hyperlipidemia,BPQ060,BPQ070,BPQ090D,BPQ100D
SEQN,,,,,,,,,,
93705,1,1,50,1,1,2,1,2,2,NaN
93706,2,NaN,NaN,NaN,NaN,2,1,2,2,NaN
93708,1,1,50,1,1,1,NaN,1,1,1
93709,1,1,71,1,1,2,1,2,2,NaN
93711,2,NaN,NaN,NaN,NaN,1,NaN,1,1,2


In [9]:
# Reduce cardio df to columns I care about
cardio_df = cardio_df[['HighBlood', 'Hyperlipidemia']]

# Map values to be human readable
condition_mapping = {1.0 : 'Yes', 2.0 : 'No', 7.0 : np.nan, 9.0 : 'No'}
cardio_df['HighBlood'] = cardio_df['HighBlood'].map(condition_mapping)
cardio_df['Hyperlipidemia'] = cardio_df['Hyperlipidemia'].map(condition_mapping)

# Check df again
cardio_df.head(5)

,HighBlood,Hyperlipidemia
SEQN,,
93705,Yes,No
93706,No,No
93708,Yes,Yes
93709,Yes,No
93711,No,Yes


In [10]:
# Merge cardio_df and demo_df into NHANES_df -- will keep adding on to this df as we go
NHANES_df = demo_df.merge(cardio_df, on = 'SEQN')
NHANES_df.head(5)

,Age,Gender,Children,HighBlood,Hyperlipidemia
SEQN,,,,,
93705,66,Female,0,Yes,No
93706,18,Male,0,No,No
93708,66,Female,0,Yes,Yes
93709,75,Female,0,Yes,No
93711,56,Male,0,No,Yes


In [11]:
# Load questionnaire/diabetes health df, which is an XPT (SAS) file. Check dataframe post-load
diab_df = pd.read_sas('C:/Users/essay/Documents/D210 PA Dataset/diabetes.XPT', index='SEQN')
diab_df.head(10)

,DIQ010,DID040,DIQ160,DIQ170,DIQ172,DIQ175A,DIQ175B,DIQ175C,DIQ175D,DIQ175E,DIQ175F,DIQ175G,DIQ175H,DIQ175I,DIQ175J,DIQ175K,DIQ175L,DIQ175M,DIQ175N,DIQ175O,DIQ175P,DIQ175Q,DIQ175R,DIQ175S,DIQ175T,DIQ175U,DIQ175V,DIQ175W,DIQ175X,DIQ180,DIQ050,DID060,DIQ060U,DIQ070,DIQ230,DIQ240,DID250,DID260,DIQ260U,DIQ275,DIQ280,DIQ291,DIQ300S,DIQ300D,DID310S,DID310D,DID320,DID330,DID341,DID350,DIQ350U,DIQ360,DIQ080
SEQN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
93703,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93704,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93705,2,NaN,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93706,2,NaN,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93707,2,NaN,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93708,3,NaN,NaN,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93709,2,NaN,1,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93711,2,NaN,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93712,2,NaN,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Rename columns to make them human-readable and more akin to those in WGU dataset.
diab_df.rename(columns = {'DIQ010': 'Diabetes'}, inplace = True)

# Reduce cardio df to columns I care about
diab_df = diab_df[['Diabetes']]

# Map values to be human readable
diabetes_mapping = {1.0 : 'Yes', 2.0 : 'No', 3.0: 'No', 7.0 : np.nan, 9.0 : 'No'}
diab_df['Diabetes'] = diab_df['Diabetes'].map(diabetes_mapping)

# Check df again
diab_df.head(5)

,Diabetes
SEQN,
93703,No
93704,No
93705,No
93706,No
93707,No


In [13]:
# Merge diab_df to NHANES_df
NHANES_df = NHANES_df.merge(diab_df, on = 'SEQN')
NHANES_df.head(5)

,Age,Gender,Children,HighBlood,Hyperlipidemia,Diabetes
SEQN,,,,,,
93705,66,Female,0,Yes,No,No
93706,18,Male,0,No,No,No
93708,66,Female,0,Yes,Yes,No
93709,75,Female,0,Yes,No,No
93711,56,Male,0,No,Yes,No


In [14]:
# Load questionnaire/conditions health df, which is an XPT (SAS) file. Check dataframe post-load
condit_df = pd.read_sas('C:/Users/essay/Documents/D210 PA Dataset/conditions.XPT', index='SEQN')
condit_df.head(10)

,MCQ010,MCQ025,MCQ035,MCQ040,MCQ050,AGQ030,MCQ053,MCQ080,MCQ092,MCD093,MCQ149,MCQ151,RHD018,MCQ160A,MCD180A,MCQ195,MCQ160N,MCD180N,MCQ160B,MCD180B,MCQ160C,MCD180C,MCQ160D,MCD180D,MCQ160E,MCD180E,MCQ160F,MCD180F,MCQ160M,MCQ170M,MCD180M,MCQ160G,MCD180G,MCQ160K,MCQ170K,MCD180K,MCQ160O,MCQ160L,MCQ170L,MCD180L,MCQ500,MCQ510A,MCQ510B,MCQ510C,MCQ510D,MCQ510E,MCQ510F,MCQ520,MCQ530,MCQ540,MCQ550,MCQ560,MCQ570,MCQ203,MCQ206,MCQ220,MCQ230A,MCD240A,MCQ230B,MCD240B,MCQ230C,MCD240C,MCQ230D,MCQ300B,MCQ300C,MCQ300A,MCQ366A,MCQ366B,MCQ366C,MCQ366D,MCQ371A,MCQ371B,MCQ371C,MCQ371D,OSQ230
SEQN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
93703,2,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93704,2,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93705,1,10,2,NaN,NaN,NaN,2,2,2,NaN,NaN,NaN,NaN,1,64,2,2,NaN,2,NaN,2,NaN,2,NaN,2,NaN,2,NaN,2,NaN,NaN,2,NaN,2,NaN,NaN,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,2,2,NaN,2,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,2,2,2,2,2,1,1,1,1,2
93706,2,NaN,NaN,NaN,NaN,NaN,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,2,2,2,2,2,2,2,2,NaN
93707,2,NaN,NaN,NaN,NaN,NaN,2,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93708,2,NaN,NaN,NaN,NaN,NaN,2,2,2,NaN,NaN,NaN,NaN,1,66,1,2,NaN,2,NaN,2,NaN,2,NaN,2,NaN,2,NaN,2,NaN,NaN,2,NaN,2,NaN,NaN,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,2,2,NaN,2,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,2,2,2,1,2,1,2,1,2,1
93709,2,NaN,NaN,NaN,NaN,NaN,2,1,2,NaN,NaN,NaN,NaN,1,47,3,2,NaN,2,NaN,2,NaN,2,NaN,2,NaN,1,60,1,2,34,2,NaN,2,NaN,NaN,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,1,1,34,2,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,2,2,1,2,2,1,1,1,1,2
93711,2,NaN,NaN,NaN,NaN,NaN,2,2,2,NaN,NaN,NaN,NaN,2,NaN,NaN,2,NaN,2,NaN,2,NaN,2,NaN,2,NaN,2,NaN,2,NaN,NaN,2,NaN,2,NaN,NaN,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,2,2,NaN,1,20,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,1,2,2,2,2,1,2,1,1,2
93712,1,5,2,NaN,NaN,NaN,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,2,1,1,2,2,1,1,1,NaN


In [15]:
# Rename columns to make them human-readable and more akin to those in WGU dataset.
condit_df.rename(columns = {'MCQ010': 'Asthma', 'MCQ080' : 'Overweight', 'MCQ160A' : 'Arthritis', 'MCQ160F': 'Stroke'}, inplace = True)

# Reduce cardio df to columns I care about
condit_df = condit_df[['Stroke', 'Overweight', 'Arthritis', 'Asthma']]

# Map values to be human readable
condit_df['Asthma'] = condit_df['Asthma'].map(condition_mapping)
condit_df['Overweight'] = condit_df['Overweight'].map(condition_mapping)
condit_df['Arthritis'] = condit_df['Arthritis'].map(condition_mapping)
condit_df['Stroke'] = condit_df['Stroke'].map(condition_mapping)

# Check df again
condit_df.head(5)

,Stroke,Overweight,Arthritis,Asthma
SEQN,,,,
93703,NaN,NaN,NaN,No
93704,NaN,NaN,NaN,No
93705,No,No,Yes,Yes
93706,NaN,No,NaN,No
93707,NaN,NaN,NaN,No


In [16]:
# Merge condit_df to NHANES_df
NHANES_df = NHANES_df.merge(condit_df, on = 'SEQN')
NHANES_df.head(5)

,Age,Gender,Children,HighBlood,Hyperlipidemia,Diabetes,Stroke,Overweight,Arthritis,Asthma
SEQN,,,,,,,,,,
93705,66,Female,0,Yes,No,No,No,No,Yes,Yes
93706,18,Male,0,No,No,No,NaN,No,NaN,No
93708,66,Female,0,Yes,Yes,No,No,No,Yes,No
93709,75,Female,0,Yes,No,No,Yes,Yes,Yes,No
93711,56,Male,0,No,Yes,No,No,No,No,No


In [17]:
# Add source column
NHANES_df['Source'] = 'CDC'

NHANES_df.head(10)

,Age,Gender,Children,HighBlood,Hyperlipidemia,Diabetes,Stroke,Overweight,Arthritis,Asthma,Source
SEQN,,,,,,,,,,,
93705,66,Female,0,Yes,No,No,No,No,Yes,Yes,CDC
93706,18,Male,0,No,No,No,NaN,No,NaN,No,CDC
93708,66,Female,0,Yes,Yes,No,No,No,Yes,No,CDC
93709,75,Female,0,Yes,No,No,Yes,Yes,Yes,No,CDC
93711,56,Male,0,No,Yes,No,No,No,No,No,CDC
93712,18,Male,2,No,No,No,NaN,No,NaN,Yes,CDC
93713,67,Male,0,No,No,No,No,No,Yes,No,CDC
93714,54,Female,1,No,Yes,Yes,No,Yes,No,No,CDC
93715,71,Male,3,No,Yes,No,No,No,Yes,No,CDC


In [18]:
# Now it's time to clean for missing data and fix some issues so that the CDC data
# and the WGU data can be compared apples to apples.
# Note that there were no missing datapoints in Age at the start, before merges performed.
NHANES_df.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 6161 entries, 93705.0 to 102956.0
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             6161 non-null   float64
 1   Gender          6161 non-null   object 
 2   Children        6161 non-null   float64
 3   HighBlood       6161 non-null   object 
 4   Hyperlipidemia  6159 non-null   object 
 5   Diabetes        6161 non-null   object 
 6   Stroke          5569 non-null   object 
 7   Overweight      6161 non-null   object 
 8   Arthritis       5569 non-null   object 
 9   Asthma          6161 non-null   object 
 10  Source          6161 non-null   object 
dtypes: float64(2), object(9)
memory usage: 577.6+ KB


In [19]:
# Deal with missing by assuming "No" if NaN
# No missing data in columns that don't house Yes or No as the choices
NHANES_df.fillna('No', inplace=True)
NHANES_df.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 6161 entries, 93705.0 to 102956.0
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             6161 non-null   float64
 1   Gender          6161 non-null   object 
 2   Children        6161 non-null   float64
 3   HighBlood       6161 non-null   object 
 4   Hyperlipidemia  6161 non-null   object 
 5   Diabetes        6161 non-null   object 
 6   Stroke          6161 non-null   object 
 7   Overweight      6161 non-null   object 
 8   Arthritis       6161 non-null   object 
 9   Asthma          6161 non-null   object 
 10  Source          6161 non-null   object 
dtypes: float64(2), object(9)
memory usage: 577.6+ KB


In [20]:
# Since WGU data doesn't include those under 18, we need to remove those datapoints from NHANES.
NHANES_df = NHANES_df[NHANES_df['Age'] > 17]
NHANES_df.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 5856 entries, 93705.0 to 102956.0
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             5856 non-null   float64
 1   Gender          5856 non-null   object 
 2   Children        5856 non-null   float64
 3   HighBlood       5856 non-null   object 
 4   Hyperlipidemia  5856 non-null   object 
 5   Diabetes        5856 non-null   object 
 6   Stroke          5856 non-null   object 
 7   Overweight      5856 non-null   object 
 8   Arthritis       5856 non-null   object 
 9   Asthma          5856 non-null   object 
 10  Source          5856 non-null   object 
dtypes: float64(2), object(9)
memory usage: 549.0+ KB


In [21]:
# Fix a couple datatype issues
NHANES_df['Age'] = NHANES_df['Age'].astype('int64')
NHANES_df['Children'] = NHANES_df['Children'].astype('int64')
NHANES_df['Gender'] = NHANES_df['Gender'].astype('category')
NHANES_df['HighBlood'] = NHANES_df['HighBlood'].astype('category')
NHANES_df['Hyperlipidemia'] = NHANES_df['Hyperlipidemia'].astype('category')
NHANES_df['Diabetes'] = NHANES_df['Diabetes'].astype('category')
NHANES_df['Stroke'] = NHANES_df['Stroke'].astype('category')
NHANES_df['Overweight'] = NHANES_df['Overweight'].astype('category')
NHANES_df['Arthritis'] = NHANES_df['Arthritis'].astype('category')
NHANES_df['Asthma'] = NHANES_df['Asthma'].astype('category')
NHANES_df['Source'] = NHANES_df['Source'].astype('category')
NHANES_df.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 5856 entries, 93705.0 to 102956.0
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   Age             5856 non-null   int64   
 1   Gender          5856 non-null   category
 2   Children        5856 non-null   int64   
 3   HighBlood       5856 non-null   category
 4   Hyperlipidemia  5856 non-null   category
 5   Diabetes        5856 non-null   category
 6   Stroke          5856 non-null   category
 7   Overweight      5856 non-null   category
 8   Arthritis       5856 non-null   category
 9   Asthma          5856 non-null   category
 10  Source          5856 non-null   category
dtypes: category(9), int64(2)
memory usage: 189.8 KB


In [22]:
# Export final version of NHANES dataset.
NHANES_df.to_csv('NHANES_clean.csv', index= False)

In [23]:
# Time to make sure "clean" WGU data is actually clean.
# Import the WGU csv file.
WGU_df = pd.read_csv('C:/Users/essay/Documents/D210 PA Dataset/medical_clean.csv', index_col = 0)

# Take a look at dataframe
WGU_df.head(10)

,Customer_id,Interaction,UID,City,State,County,Zip,Lat,Lng,Population,Area,TimeZone,Job,Children,Age,Income,Marital,Gender,ReAdmis,VitD_levels,Doc_visits,Full_meals_eaten,vitD_supp,Soft_drink,Initial_admin,HighBlood,Stroke,Complication_risk,Overweight,Arthritis,Diabetes,Hyperlipidemia,BackPain,Anxiety,Allergic_rhinitis,Reflux_esophagitis,Asthma,Services,Initial_days,TotalCharge,Additional_charges,Item1,Item2,Item3,Item4,Item5,Item6,Item7,Item8
CaseOrder,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,C412403,8cd49b13-f45a-4b47-a2bd-173ffa932c2f,3a83ddb66e2ae73798bdf1d705dc0932,Eva,AL,Morgan,35621,34,-87,2951,Suburban,America/Chicago,"Psychologist, sport and exercise",1,53,86576,Divorced,Male,No,19,6,0,0,No,Emergency Admission,Yes,No,Medium,No,Yes,Yes,No,Yes,Yes,Yes,No,Yes,Blood Work,11,3727,17939,3,3,2,2,4,3,3,4
2,Z919181,d2450b70-0337-4406-bdbb-bc1037f1734c,176354c5eef714957d486009feabf195,Marianna,FL,Jackson,32446,31,-85,11303,Urban,America/Chicago,Community development worker,3,51,46806,Married,Female,No,19,4,2,1,No,Emergency Admission,Yes,No,High,Yes,No,No,No,No,No,No,Yes,No,Intravenous,15,4193,17613,3,4,3,4,4,4,3,3
3,F995323,a2057123-abf5-4a2c-abad-8ffe33512562,e19a0fa00aeda885b8a436757e889bc9,Sioux Falls,SD,Minnehaha,57110,44,-97,17125,Suburban,America/Chicago,Chief Executive Officer,3,53,14370,Widowed,Female,No,18,4,1,0,No,Elective Admission,Yes,No,Medium,Yes,No,Yes,No,No,No,No,No,No,Blood Work,5,2434,17505,2,4,4,4,3,4,3,3
4,A879973,1dec528d-eb34-4079-adce-0d7a40e82205,cd17d7b6d152cb6f23957346d11c3f07,New Richland,MN,Waseca,56072,44,-94,2162,Suburban,America/Chicago,Early years teacher,0,78,39741,Married,Male,No,17,4,1,0,No,Elective Admission,No,Yes,Medium,No,Yes,No,No,No,No,No,Yes,Yes,Blood Work,2,2128,12993,3,5,5,3,4,5,5,5
5,C544523,5885f56b-d6da-43a3-8760-83583af94266,d2f0425877b10ed6bb381f3e2579424a,West Point,VA,King William,23181,38,-77,5287,Rural,America/New_York,Health promotion specialist,1,22,1210,Widowed,Female,No,17,5,0,2,Yes,Elective Admission,No,No,Low,No,No,No,Yes,No,No,Yes,No,No,CT Scan,1,2113,3717,2,1,3,3,5,3,4,3
6,S543885,e3b0a319-9e2e-4a23-8752-2fdc736c30f4,03e447146d4a32e1aaf75727c3d1230c,Braggs,OK,Muskogee,74423,36,-95,981,Urban,America/Chicago,Corporate treasurer,3,76,82000,Never Married,Male,No,20,6,0,0,No,Observation Admission,No,No,Medium,Yes,Yes,Yes,No,Yes,No,Yes,No,No,Blood Work,6,2637,12743,4,5,4,4,3,5,4,6
7,E543302,2fccb53e-bd9a-4eaa-a53c-9dfc0cb83f94,e4884a42ba809df6a89ded6c97f460d4,Thompson,OH,Geauga,44086,42,-81,2558,Rural,America/New_York,Hydrologist,0,50,10456,Never Married,Male,No,15,6,0,0,No,Emergency Admission,Yes,No,Low,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes,No,Intravenous,9,3695,16816,4,3,3,2,3,4,5,5
8,K477307,ab634508-dd8c-42e5-a4e4-d101a46f2431,5f78b8699d1aa9b950b562073f629ca2,Strasburg,VA,Shenandoah,22641,39,-78,479,Urban,America/New_York,Psychiatric nurse,7,40,38319,Divorced,Female,No,20,7,2,0,No,Observation Admission,No,No,Medium,Yes,No,No,No,No,No,No,No,No,Intravenous,14,3021,6931,1,2,2,5,4,2,4,2
9,Q870521,67b386eb-1d04-4020-9474-542a09d304e3,e8e016144bfbe14974752d834f530e26,Panama City,FL,Bay,32404,30,-86,40029,Urban,America/Chicago,Computer games developer,0,48,55586,Widowed,Male,No,20,6,3,0,No,Emergency Admission,No,No,Low,Yes,No,No,Yes,No,No,No,No,No,Intravenous,6,2968,8363,3,3,2,3,3,3,4,2


In [24]:
# Check datatypes for problems
WGU_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 1 to 10000
Data columns (total 49 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Customer_id         10000 non-null  object 
 1   Interaction         10000 non-null  object 
 2   UID                 10000 non-null  object 
 3   City                10000 non-null  object 
 4   State               10000 non-null  object 
 5   County              10000 non-null  object 
 6   Zip                 10000 non-null  int64  
 7   Lat                 10000 non-null  float64
 8   Lng                 10000 non-null  float64
 9   Population          10000 non-null  int64  
 10  Area                10000 non-null  object 
 11  TimeZone            10000 non-null  object 
 12  Job                 10000 non-null  object 
 13  Children            10000 non-null  int64  
 14  Age                 10000 non-null  int64  
 15  Income              10000 non-null  float64
 16  Mari

In [25]:
#Clean datatypes up using code from D206 PA. 
#[In-Text Citation: (Nelson, 2023).]

# Convert Zip to string from integer.
WGU_df['Zip'] = WGU_df['Zip'].astype('str')
# Add leading zeros using zfill()
WGU_df['Zip'] = WGU_df['Zip'].str.zfill(5)
# Identify columns that can be converted all at once to category datatype using for loop.
category_cols = WGU_df[['Area', 'Marital', 'Initial_admin', 'Complication_risk', 'Services', 'ReAdmis',
                         'Soft_drink', 'HighBlood', 'Stroke', 'Arthritis', 'Diabetes', 'Hyperlipidemia', 'BackPain',
                         'Allergic_rhinitis', 'Reflux_esophagitis', 'Asthma']]
# Will do Item# columns later since they require an order. Timezone needs a dict written, will do that later too.
for col in category_cols:
    WGU_df[col] = WGU_df[col].astype('category')
# Convert gender to category datatype.
WGU_df['Gender'] = WGU_df['Gender'].astype('category')
# Convert Overweight to category datatype.
WGU_df['Overweight'] = WGU_df['Overweight'].astype('category')
# Convert Anxiety to category datatype.
WGU_df['Anxiety'] = WGU_df['Anxiety'].astype('category')
# Convert Job to category datatype.
WGU_df['Job'] = WGU_df['Job'].astype('category')

# Create ordered categories for Item# variables. 8 is "least important" and 1 is "most important"
survey_scores = CategoricalDtype(categories=['8', '7', '6', '5', '4', '3', '2', '1'], ordered=True)
# Identify columns that need to become ordered categorical
ord_cat_cols = WGU_df[['Item1', 'Item2', 'Item3', 'Item4', 'Item5', 'Item6', 'Item7', 'Item8']]
# Create for loop to convert columns above to string (issues if not string first,) then ordered categorical datatype.
for col in ord_cat_cols:
    WGU_df[col] = WGU_df[col].astype('str')
    WGU_df[col] = WGU_df[col].astype(survey_scores)
# convert Initial_days to an integer.
WGU_df['Initial_days'] = WGU_df['Initial_days'].astype('int64')
# Write dictionary for option reduction for Timezone column
mapping_timezone = {'America/Puerto_Rico' : 'Atlantic',
          'America/New_York' : 'Eastern',
          'America/Detroit' : 'Eastern',
          'America/Indiana/Indianapolis' : 'Eastern',
          'America/Indiana/Vevay' : 'Eastern',
          'America/Indiana/Vincennes' : 'Eastern',
          'America/Kentucky/Louisville' : 'Eastern',
          'America/Toronto' : 'Eastern',
          'America/Indiana/Marengo' : 'Eastern',
          'America/Indiana/Winamac' : 'Eastern',
          'America/Chicago' : 'Central',
          'America/Menominee' : 'Central',
          'America/Indiana/Knox' : 'Central',
          'America/Indiana/Tell_City' : 'Central',
          'America/North_Dakota/Beulah' : 'Central',
          'America/North_Dakota/New_Salem' : 'Central',
          'America/Denver' : 'Mountain',
          'America/Boise' : 'Mountain',
          'America/Phoenix' : 'Mountain',
          'America/Los_Angeles' : 'Pacific',
          'America/Nome' : 'Alaskan',
          'America/Anchorage' : 'Alaskan',
          'America/Sitka' : 'Alaskan',
          'America/Yakutat' : 'Alaskan',
          'America/Adak' : 'Hawaiian',
          'Pacific/Honolulu' : 'Hawaiian'
          }
# Use dictionary to convert timezone options.
WGU_df.TimeZone.replace(mapping_timezone, inplace=True)
# Convert timezone to category datatype.
WGU_df['TimeZone'] = WGU_df['TimeZone'].astype('category')
# Round TotalCharge to 2 decimal places
WGU_df['TotalCharge'] = WGU_df.TotalCharge.round(2)
# Round Additional_charges to 2 decimal places
WGU_df['Additional_charges'] = WGU_df.Additional_charges.round(2)

In [26]:
# Check if changes took
WGU_df.head(10)

,Customer_id,Interaction,UID,City,State,County,Zip,Lat,Lng,Population,Area,TimeZone,Job,Children,Age,Income,Marital,Gender,ReAdmis,VitD_levels,Doc_visits,Full_meals_eaten,vitD_supp,Soft_drink,Initial_admin,HighBlood,Stroke,Complication_risk,Overweight,Arthritis,Diabetes,Hyperlipidemia,BackPain,Anxiety,Allergic_rhinitis,Reflux_esophagitis,Asthma,Services,Initial_days,TotalCharge,Additional_charges,Item1,Item2,Item3,Item4,Item5,Item6,Item7,Item8
CaseOrder,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,C412403,8cd49b13-f45a-4b47-a2bd-173ffa932c2f,3a83ddb66e2ae73798bdf1d705dc0932,Eva,AL,Morgan,35621,34,-87,2951,Suburban,Central,"Psychologist, sport and exercise",1,53,86576,Divorced,Male,No,19,6,0,0,No,Emergency Admission,Yes,No,Medium,No,Yes,Yes,No,Yes,Yes,Yes,No,Yes,Blood Work,10,3727,17939,3,3,2,2,4,3,3,4
2,Z919181,d2450b70-0337-4406-bdbb-bc1037f1734c,176354c5eef714957d486009feabf195,Marianna,FL,Jackson,32446,31,-85,11303,Urban,Central,Community development worker,3,51,46806,Married,Female,No,19,4,2,1,No,Emergency Admission,Yes,No,High,Yes,No,No,No,No,No,No,Yes,No,Intravenous,15,4193,17613,3,4,3,4,4,4,3,3
3,F995323,a2057123-abf5-4a2c-abad-8ffe33512562,e19a0fa00aeda885b8a436757e889bc9,Sioux Falls,SD,Minnehaha,57110,44,-97,17125,Suburban,Central,Chief Executive Officer,3,53,14370,Widowed,Female,No,18,4,1,0,No,Elective Admission,Yes,No,Medium,Yes,No,Yes,No,No,No,No,No,No,Blood Work,4,2434,17505,2,4,4,4,3,4,3,3
4,A879973,1dec528d-eb34-4079-adce-0d7a40e82205,cd17d7b6d152cb6f23957346d11c3f07,New Richland,MN,Waseca,56072,44,-94,2162,Suburban,Central,Early years teacher,0,78,39741,Married,Male,No,17,4,1,0,No,Elective Admission,No,Yes,Medium,No,Yes,No,No,No,No,No,Yes,Yes,Blood Work,1,2128,12993,3,5,5,3,4,5,5,5
5,C544523,5885f56b-d6da-43a3-8760-83583af94266,d2f0425877b10ed6bb381f3e2579424a,West Point,VA,King William,23181,38,-77,5287,Rural,Eastern,Health promotion specialist,1,22,1210,Widowed,Female,No,17,5,0,2,Yes,Elective Admission,No,No,Low,No,No,No,Yes,No,No,Yes,No,No,CT Scan,1,2113,3717,2,1,3,3,5,3,4,3
6,S543885,e3b0a319-9e2e-4a23-8752-2fdc736c30f4,03e447146d4a32e1aaf75727c3d1230c,Braggs,OK,Muskogee,74423,36,-95,981,Urban,Central,Corporate treasurer,3,76,82000,Never Married,Male,No,20,6,0,0,No,Observation Admission,No,No,Medium,Yes,Yes,Yes,No,Yes,No,Yes,No,No,Blood Work,5,2637,12743,4,5,4,4,3,5,4,6
7,E543302,2fccb53e-bd9a-4eaa-a53c-9dfc0cb83f94,e4884a42ba809df6a89ded6c97f460d4,Thompson,OH,Geauga,44086,42,-81,2558,Rural,Eastern,Hydrologist,0,50,10456,Never Married,Male,No,15,6,0,0,No,Emergency Admission,Yes,No,Low,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes,No,Intravenous,9,3695,16816,4,3,3,2,3,4,5,5
8,K477307,ab634508-dd8c-42e5-a4e4-d101a46f2431,5f78b8699d1aa9b950b562073f629ca2,Strasburg,VA,Shenandoah,22641,39,-78,479,Urban,Eastern,Psychiatric nurse,7,40,38319,Divorced,Female,No,20,7,2,0,No,Observation Admission,No,No,Medium,Yes,No,No,No,No,No,No,No,No,Intravenous,14,3022,6931,1,2,2,5,4,2,4,2
9,Q870521,67b386eb-1d04-4020-9474-542a09d304e3,e8e016144bfbe14974752d834f530e26,Panama City,FL,Bay,32404,30,-86,40029,Urban,Central,Computer games developer,0,48,55586,Widowed,Male,No,20,6,3,0,No,Emergency Admission,No,No,Low,Yes,No,No,Yes,No,No,No,No,No,Intravenous,6,2968,8363,3,3,2,3,3,3,4,2


In [27]:
WGU_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 1 to 10000
Data columns (total 49 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   Customer_id         10000 non-null  object  
 1   Interaction         10000 non-null  object  
 2   UID                 10000 non-null  object  
 3   City                10000 non-null  object  
 4   State               10000 non-null  object  
 5   County              10000 non-null  object  
 6   Zip                 10000 non-null  object  
 7   Lat                 10000 non-null  float64 
 8   Lng                 10000 non-null  float64 
 9   Population          10000 non-null  int64   
 10  Area                10000 non-null  category
 11  TimeZone            10000 non-null  category
 12  Job                 10000 non-null  category
 13  Children            10000 non-null  int64   
 14  Age                 10000 non-null  int64   
 15  Income              10000 non-null  

In [28]:
# Reduce WGU dataset to the same columns we have for the NHANES dataset.
WGU_df = WGU_df[['Age','Gender','Children','HighBlood','Hyperlipidemia','Diabetes','Stroke','Overweight','Arthritis','Asthma']]


In [29]:
# NHANES data lists people over 80 as "80," so we'll do the same here.
WGU_df.loc[WGU_df['Age'] > 80, 'Age'] = 80

#Check dataframe
WGU_df.head(5)

,Age,Gender,Children,HighBlood,Hyperlipidemia,Diabetes,Stroke,Overweight,Arthritis,Asthma
CaseOrder,,,,,,,,,,
1,53,Male,1,Yes,No,Yes,No,No,Yes,Yes
2,51,Female,3,Yes,No,No,No,Yes,No,No
3,53,Female,3,Yes,No,Yes,No,Yes,No,No
4,78,Male,0,No,No,No,Yes,No,Yes,Yes
5,22,Female,1,No,Yes,No,No,No,No,No


In [30]:
WGU_df.Age.describe()

count   10000
mean       53
std        20
min        18
25%        36
50%        53
75%        71
max        80
Name: Age, dtype: float64

In [31]:
# Add source column like we did for NHANES.
WGU_df['Source'] = 'WGU'

# Export final version of WGU dataset.
WGU_df.to_csv('WGU_clean.csv', index= False)

In [32]:
# Create final, combined dataframe that is easy for Tableau to use without funky relationships.
combined_df = pd.concat([WGU_df, NHANES_df], ignore_index=True)
# Check result
combined_df

,Age,Gender,Children,HighBlood,Hyperlipidemia,Diabetes,Stroke,Overweight,Arthritis,Asthma,Source
0,53,Male,1,Yes,No,Yes,No,No,Yes,Yes,WGU
1,51,Female,3,Yes,No,No,No,Yes,No,No,WGU
2,53,Female,3,Yes,No,Yes,No,Yes,No,No,WGU
3,78,Male,0,No,No,No,Yes,No,Yes,Yes,WGU
4,22,Female,1,No,Yes,No,No,No,No,No,WGU
...,...,...,...,...,...,...,...,...,...,...,...
15851,64,Male,0,Yes,Yes,No,Yes,Yes,Yes,No,CDC
15852,70,Female,0,No,No,No,No,No,No,No,CDC
15853,42,Male,0,No,Yes,No,No,Yes,No,Yes,CDC
15854,41,Female,4,No,No,No,No,Yes,No,No,CDC


In [33]:
# Export final version of combined dataset for further analysis in Tableau
combined_df.to_csv('NHANES_plus_WGU_data.csv', index= False)

## A2. Dashboard Installation

Double-click Tableau icon on desktop to open Tableau.

### Loading Data
On the blue, left pane, click "Text File" under the "To a File" subsection.
Locate the external dataset by browsing through the folders on the local directory. Select the file and click "Open." This brings you to a screen where you can preview the data. Click "Sheet 1" at the bottom of the screen. Double-click "Sheet 1" and change the name to "Age Boxplot." This is the first visualization we will make after we do some preprocessing.

### Preprocessing Dataset
Tableau automatically tries to assign variables appropriate datatypes and place them correctly either as dimensions (above the gray line on the Data pane on the left) or measures (below the gray line.) Sometimes, we need to make modifications, as Tableau does make mistakes from time to time.

However, all the Yes/No condition variables appear to be correctly assigned the string datatype and are dimensions, while Age and Children have been assigned a numerical datatype and are appropriately measures. Thus, no preprocessing is needed before making our visualizations.

### Age Boxplot Visualization
Drag "Age" from the Data pane to the Rows shelf. Click "Analysis" on the white ribbon at the top and uncheck "aggregate measures." Click "Show Me" in the top right and select the boxplot icon to create a box plot. Drag "Source" from the Data pane to the Columns shelf to separate the data by source (i.e. WGU vs CDC data.) This gives you two box plots.

Click on "Analytics" on the left to switch from the Data pane to the Analytics pane. Drag "Average Line" to the visualization, and drop it on the tile called "Cell." This adds average lines that are specific to each boxplot.

Click the gray part of the boxplot and click "Format" on the box that pops up. Change the "Fill" dropdown on the left to "Orange" because it stands out more than the gray. Use the sliding switch to change the opacity to 50%. Click the "x" on the Format section so you can see the Data pane again.

Right-click "Gender" in the Data pane and click "Show Filter."

Right-click "Source" at the top of the visualization (a label) and click "Hide Field Labels for Columns."

### Age Histogram Visualization
Click the "add new worksheet" icon at the bottom and name it "Age Histogram." Select "Age" from the Data pane and then click "Show Me." In the Show Me menu, select the histogram icon. Now, drag "Source" from the Data pane to the columns shelf. In the Data pane, right-click the newly-made "Age (bin)" data field. Click "Edit" in the menu, then change "Size of bins" to 3. Click "OK." Right-click the y-axis and click "Edit Axis." Change the axis title to "Number of People" and exit.

Right-click "Source" at the top of the visualization (a label) and click "Hide Field Labels for Columns."

Right-click "Arthritis" in the Data pane and click "Show Filter." Repeat for "Asthma," "Diabetes," "High Blood," "Hyperlipidemia," "Overweight," and "Stroke." For each filter on the right, click the small black down-arrow in the top right of the filter and select "Single Value (dropdown)." If they default to being filtered on "No," clear the filter with the funnel-with-an-x icon in the top right of the filter in question.

Double-click the tooltip marks card and edit the text in the text box. Replace "Count of Age" with "# of People". Delete "Source:	< Source >." Click "OK."

### Average # of Children Bar Chart Visualization
Click the "add new worksheet" icon at the bottom and name it "Avg # Children Bar Graph." Drag "Source" from the Data pane to the columns shelf. Drag "Age (bin)" from the Data pane to the columns shelf. Drag "Children" from the Data pane to the columns shelf, then right-click it, hover over "Measure (Sum)" and select "Average" from the menu that appears. Press and hold Ctrl on the keyboard while dragging "AVG(Children)" from the rows shelf to the label card. This adds labels over the bars. Right-click the y-axis and click "Edit Axis." Change the axis title to "Avg. # of Children" and exit.

Right-click "Source" at the top of the visualization (a label) and click "Hide Field Labels for Columns."

Double-click the tooltip marks card and edit the text in the text box. Delete "Source:	< Source >." Change "Avg. Children" to "Avg. # of Children." Click "OK."

### Gender Pie Chart Visualization
Click the "add new worksheet" icon at the bottom and name it as "Gender Pie Chart." Use the dropdown on the marks card to change the visualization type to "Pie." Drag "Gender" from the Data pane to the color marks card. Drag "Gender" from the Data pane again, but this time drag it to the angle marks card. Both of these will show up underneath the different marks cards. Right-click the "Gender" bar with the angle icon and hover over "Measure," then select "Count." Right-click the same "Gender" bar again (which now says "CNT(Gender)" and hover over "Quick Table Calculation." From the menu that appears, click "Percent of Total." Right-click "CNT(Gender)" once more and click "Edit Table Calculation." In the pop-up, under the "Compute Using" section, select "Table (down)" and exit. Hold Ctrl on your keyboard and, at the same time, drag the "CNT(Gender)" bar up to the label marks card.

Drag "Source" from the Data pane to the columns shelf. Right-click "Source" at the top of the visualization (a label) and click "Hide Field Labels for Columns."

Click the "Color" marks card and select "Edit Colors" from the pop-up that appears. In the menu that appears, use the "Select Color Palette" drop down to change the color palette to "Color Blind." Click "Assign Palette" and then click "OK."

Double-click the tooltip marks card and edit the text in the text box. Delete "Source:	< Source >." Change "% of Total Count of Gender along Table (Down)" to "% of Total." Click "OK.

### KPI - Overweight
Click the "add new worksheet" icon at the bottom and name it as "KPI Overweight." Drag "Source" from the Data pane to the columns shelf. Drag "Overweight" from the Data pane to the rows shelf. Drag "Overweight" from the Data pane again, but this time to the Text card. Right-click the "Overweight" bar that appears on the marks card, hover over "Measure," and select "Count." Right-click this bar again, hover over "Quick Table Calculation," and from the menu that appears, click "Percent of Total." Right-click that same bar once more and click "Edit Table Calculation." In the pop-up, under the "Compute Using" section, select "Table (down)" and exit.

On the visualization, right-click the "No" row-label and select "Hide." Right-click the title "Overweight" on the visualization and click "Hide Field Labels for Rows." Right-click "Source" at the top of the visualization (a label) and click "Hide Field Labels for Columns." Right-click the "Yes" row-label and click "Show Header" to uncheck it. Right-click the title and select "Edit Title." Center it using the three-lines icon that looks centered. Click "OK." On the gray ribbon across the top of the program, select the down-arrow next to a dropdown that currently says "Standard." Change this to "Fit Width." Right-click the datapoint in the visualization and click "Format." Click the alignment icon (which looks like a stack of lines) on the formatting pane, which covers the data pane for now. Click the first dropdown under the "Default" section. A small pop-up appears. In the "Horizontal" section, select the centered-lines icon.

Double-click the tooltip marks card and edit the text in the text box. Delete "Source:	< Source >." Change "% of Total Count of Overweight along Table (Down)" to "% of Total."

### KPI - Overweight Times Higher
Click the "add new worksheet" icon at the bottom and name it as "KPI Overweight Times Higher." Right-click "Overweight" on the Data pane, hover over "Create," then select "Calculated Field." In the box that pops up, name the calculation "# CDC Overweight." In the text box on this pop-up, type "SUM(IF[Overweight] = "Yes" AND [Source] = "CDC" THEN 1 ELSE 0 END)" and click "OK." Right-click "Overweight" on the Data pane again, hover over "Create," then select "Calculated Field." In the box that pops up, name the calculation "# WGU Overweight." In the text box on this pop-up, type "SUM(IF[Overweight] = "Yes" AND [Source] = "WGU" THEN 1 ELSE 0 END)" and click "OK." Right-click "# CDC Overweight" and click "duplicate." Then, right-click the newly created field and click "Edit." Replace the formula in the box with "SUM(IF[Source] = "CDC" THEN 1 ELSE 0 END)". Rename it to "# CDC Patients." Right-click "# WGU Overweight" and click "duplicate." Then, right-click the newly created field and click "Edit." Replace the formula in the box with "SUM(IF[Source] = "WGU" THEN 1 ELSE 0 END)". Rename it to "# WGU Patients." Right-click "Overweight" on the Data pane again, hover over "Create," then select "Calculated Field." In the box that pops up, name the calculation "WGU Overweight Times Higher." In the text box on this pop-up, type "([# WGU Overweight]/[# WGU Patients])/
([# CDC Overweight]/[# CDC Patients])" and click "OK." From the Data pane, drag "WGU Overweight Times Higher" to the text marks card.

Right-click the title and select "Edit Title." Type "Overweight" then a carriage return, and "Times Higher than CDC." Center it using the three-lines icon that looks centered. Click "OK." On the gray ribbon across the top of the program, select the down-arrow next to a dropdown that currently says "Standard." Change this to "Fit Width." Right-click the datapoint in the visualization and click "Format." Click the alignment icon (which looks like a stack of lines) on the formatting pane, which covers the data pane for now. Click the first dropdown under the "Default" section. A small pop-up appears. In the "Horizontal" section, select the centered-lines icon. Click on the small "A" to go back to the Format Font screen. Under the "Default" section, click the first dropdown, which is labeled "Worksheet." Use the dropdown to change the font to Tableau Bold, and the font size dropdown to change the font size to 14. Click one of the blue-colored squares to change the font color to a nice blue.

### KPI - High Blood Pressure
Click the "add new worksheet" icon at the bottom and name it as "KPI High Blood Pressure." Drag "Source" from the Data pane to the columns shelf. Drag "High Blood" from the Data pane to the rows shelf. Drag "High Blood" from the Data pane again, but this time to the Text card. Right-click the "High Blood" bar that appears on the marks card, hover over "Measure," and select "Count." Right-click this bar again, hover over "Quick Table Calculation," and from the menu that appears, click "Percent of Total." Right-click that same bar once more and click "Edit Table Calculation." In the pop-up, under the "Compute Using" section, select "Table (down)" and exit.

On the visualization, right-click the "No" row-label and select "Hide." Right-click the title "High Blood" on the visualization and click "Hide Field Labels for Rows." Right-click "Source" at the top of the visualization (a label) and click "Hide Field Labels for Columns." Right-click the "Yes" row-label and click "Show Header" to uncheck it. Right-click the title and select "Edit Title." Center it using the three-lines icon that looks centered. Click "OK." On the gray ribbon across the top of the program, select the down-arrow next to a dropdown that currently says "Standard." Change this to "Fit Width." Right-click the datapoint in the visualization and click "Format." Click the alignment icon (which looks like a stack of lines) on the formatting pane, which covers the data pane for now. Click the first dropdown under the "Default" section. A small pop-up appears. In the "Horizontal" section, select the centered-lines icon.

Double-click the tooltip marks card and edit the text in the text box. Delete "Source:	< Source >." Change "% of Total Count of High Blood along Table (Down)" to "% of Total."

### KPI - High BP Times Higher
Click the "add new worksheet" icon at the bottom and name it as "KPI High BP Times Higher." Right-click "High Blood" on the Data pane, hover over "Create," then select "Calculated Field." In the box that pops up, name the calculation "# CDC Overweight." In the text box on this pop-up, type "SUM(IF[High Blood] = "Yes" AND [Source] = "CDC" THEN 1 ELSE 0 END)" and click "OK." Right-click "High Blood" on the Data pane again, hover over "Create," then select "Calculated Field." In the box that pops up, name the calculation "# WGU Overweight." In the text box on this pop-up, type "SUM(IF[High Blood] = "Yes" AND [Source] = "WGU" THEN 1 ELSE 0 END)" and click "OK." Right-click "High Blood" on the Data pane again, hover over "Create," then select "Calculated Field." In the box that pops up, name the calculation "WGU High Blood Times Higher." In the text box on this pop-up, type "(([# WGU High Blood]/[# WGU Patients])/
([# CDC High Blood]/[# CDC Patients])" and click "OK." From the Data pane, drag "WGU High BP Times Higher" to the text marks card.

Right-click the title and select "Edit Title." Type "High BP" then a carriage return, and "Times Higher than CDC." Center it using the three-lines icon that looks centered. Click "OK." On the gray ribbon across the top of the program, select the down-arrow next to a dropdown that currently says "Standard." Change this to "Fit Width." Right-click the datapoint in the visualization and click "Format." Click the alignment icon (which looks like a stack of lines) on the formatting pane, which covers the data pane for now. Click the first dropdown under the "Default" section. A small pop-up appears. In the "Horizontal" section, select the centered-lines icon. Click on the small "A" to go back to the Format Font screen. Under the "Default" section, click the first dropdown, which is labeled "Worksheet." Use the dropdown to change the font to Tableau Bold, and the font size dropdown to change the font size to 14. Click one of the blue-colored squares to change the font color to a nice blue.

### Making the Dashboard
Click the "add new dashboard" icon at the bottom and name it as "Medical Data Dashboard." On the Dashboard pane on the left, change the dropdown under the section called "Size" to "Automatic." From the Dashboard pane, drag every sheet from the "Sheets" section to the dashboard and arrange them in a way that makes sense and is visually pleasing. From the Dashboard pane, at the bottom, find and drag "Horizontal Container" to the top of the dashboard. You'll know when to drop it when the top half of the dashboard is gray. Resize it as appropriate.

From the Dashboard pane, drag "Text" to this new container. This adds a text box we can use for a title. A box will pop up. Type "Our Patients: How Can We Help?" into the text field, then change the font to "Tableau Bold" using the leftmost dropdown. In the dropdown right next to this one, change the fontsize to 26 and use the color dropdown to change the color to blue to match the rest of the dashboard. Click "OK."

Right-click the "Age Histogram" title and click "Hide Title." Right-click the "Avg # Children Bar Graph" title and click "Hide Title." Right-click the "Gender Pie Chart" title and click "Hide Title." Right-click the "Age Boxplot" title and click "Hide Title." Right-click the "KPI Overweight" title and click "Hide Title." Right-click the "KPI High Blood Pressure" title and click "Hide Title."

Click the Gender legend on the right side where the filters are and click the little down-arrow. This opens a menu. From the menu, click "Floating" and drag the resulting legend near the pie charts. Click the bar containing the rest of the filters so that it's selected. Click the small down-arrow and select "Floating." Select the now floating container and click the small down-arrow again, but this time select "Remove Container." Drag the now separate filters to the horizonal container at the top of the dashboard and arrange them in a visually pleasing way. For each filter, select it and click the small down-arrow, hover over "Apply to Worksheets," then click "All Using This Data Source." Select the pie chart visualization and click the little "funnel" icon to make the visualization able to be used as a filter for the whole dashboard.

### Optimizing Different Devices
Click "Device Preview" on the dashboard pane. Use the arrows on the selections above the dashboard to change between various desktop, tablet, and phone views and sizes. For each one that you want to optimize, click the button that says "Add Desktop Layout" or "Add Phone Layout" or "Add Tablet Layout." On the dashboard pane, you will now have a tab under the top section for each device. Optimize the viewing experience for each of these devices by clicking the tab for the device and resizing the visuals on the dashboard by dragging them to appropriate sizes and locations.

### Publishing the Workbook
On the white ribbon across the top of the program, click "Server," then hover over "Tableau Public," then click "Save to Tableau Public." A login screen will pop up. Log in. Create a data extract if necessary when prompted to do so by clicking "Create Data Extract." Click "Save Settings." A system dialog will pop up so you can save the .hyper file. Do so by hitting "Save." If you had to create a data extract, on the white ribbon across the top of the program, click "Server" again, then hover over "Tableau Public," then click "Save to Tableau Public." This will actually publish it this time.

### Workbook Link
In order to view this dashboard without Tableau Desktop, you may click the following link to be taken to its Tableau Public version: https://public.tableau.com/views/D210PA_MichelleNelsonTableau/MedicalDataDashboard?:language=en-US&publish=yes&:sid=&:display_count=n&:origin=viz_share_link

## A3. Navigation

There are two ways to navigate the dashboard: by using the visualizations or by using the explicit interactive filters.

Using the visualizations to filter is actually fairly limited. While I would have liked to make every visualization usable as a filter, some visualizations just didn't prove too useful as a filter. This was the case with the histograms, bar charts, box plots, and the KPIs. Thus, these particular visualizations will not filter the whole dashboard when interacted with. However, the pie charts *can* be utilized as a filter. Clicking on any one slice of the pie (or using Ctrl+click to select multiple slices at a time) can be used to filter the entire dashboard. For example, you could hold Ctrl on your keyboard and click the female sections of the two pies to slice the whole dashboard and show only data on female patients. In addition to this, the pie chart visualization can be used to filter on data source-- the CDC or WGU-- by clicking the title "CDC" or "WGU" above each pie chart. This filters the whole dashboard by data source. To remove a filter applied from a visualization, one only needs to click outside the previously made selection to clear it.

I have also included some more explicit filters for slicing and dicing of the data. These are located in the top right corner of the dashboard. These filters default to showing all the data. The filter for gender is a series of checkboxes, so multiple selections can be made at once. To use this filter, simply check the genders you wish to slice the data by, or uncheck the ones you don't. To clear the filter--and any such filter I will discuss from here on out-- you can click the funnel-with-an-x icon in the top right of the filter. The rest of the filters are for various conditions, all of which are single selection dropdowns. To use these filters to slice the data by people with or without a certain condition, simply click the small down-arrow on the dropdown bar, select the selection you want (for example, selecting "Yes" on the Diabetes filter filters the dashboard for people with diabetes,) and watch the data transform. Using these filters, you can even apply multiple filters at once.

One last thing to note is that hovering over any piece of a visualization will often provide you a "tooltip," or a little popup box that gives you more information about the section of a visualization you are looking at. For example, if you hover over the first bin of the CDC histogram, a box will appear that helpfully tells you exactly how many people are in that bin: 363. This helps remove guesswork in reading the y-axis.

## B. Panopto
The link to my Panopto is as follows: https://wgu.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=e870a6a7-1153-4782-a32d-b15b0187090f

## C1. Alignment with Business Need
WGU's medical data dictionary signifies a need to address readmission rates, since excessive readmission can result in monetary penalties for the hospital system. Because I struggled to find a dataset that could be used to compare the WGU hospital system to the country's hospital readmission rates, I was forced to take a broader approach than might be anticipated by the audience.

Instead of trying to benchmark the WGU hospital system's readmission rate against national averages, I instead analyzed patient demographics and health conditions as compared to CDC data that respresents the country as a whole. Intuitively speaking, being overweight and having high blood pressure are both complicating factors during hospital stays and could lead to longer stays or readmissions. In addition, older patients are inherently more at risk. Ideally, prior analysis using a machine learning technique would back up this statement of intuition. Thus, analyzing how WGU's patients compare to the nation as a whole in terms of these two condition factors and patient age would be beneficial. With this analysis, the WGU hospital system can be proactive about how it works up patients because it has knowledge about the averages of its patient base compared to the national averages.

The dashboard allows for exploration of patient age and gender using a histogram and a boxplot, respectively. Gender is primarily present for slicing purposes, in case there are trends in high blood pressure or overweight status that are different based on gender. Using the histogram to view the WGU hospital system's patient age distribution may help inform WGU so it can make effective decisions regarding where to allocate resources, such as geriatric or pediatric units, that may prevent readmissions.

The two KPIs, which both signify how many times higher than the national average WGU's patient base is for each of two conditions, provide in the form of a single number how much WGU's patient base is (in most cases,) at higher risk than the rest of the country. Having one or both of the conditions highlighted, as aforementioned, is a complicating factor in a hospital stay and could lead to readmission, which the hospital must avoid. Executives of the WGU hospital system can use these KPIs to make informed decisions regarding what programs may need to be started in order to be proactive and prevent these two preventable conditions, which, in turn, would allow the lowering of readmission rates.

## C2. Exernal Dataset Justification
The external NHANES dataset serves primarily to provide a national benchmark to which I can to which I can compare the WGU hospital system's patient base. Should WGU's patient base have risk factors several times higher than the national average, WGU can use this information to take preventative action in the form of new departments or programs.

The NHANES dataset is massive, with hundreds of columns containing information I have no way of comparing to the WGU dataset. Thus, when I assembled my subset of it, I kept columns from the NHANES dataset that I could directly compare to the WGU dataset for benchmarking purposes. These columns included: age, gender, number of children, arthritis, asthma, diabetes, high blood pressure, hyperlipidemia, overweight, and stroke. These all were present in more-or-less the same format in the WGU dataset.

The two conditions I ended up turning into KPIs, high blood pressure and overweight status, both were diagnosed at rates in the WGU patient base that were strikingly higher than the CDC's patient base. One to two times higher, in fact. Without the external dataset, I would have nothing to which I could compare WGU's condition rates. I would not be able to gain the insight that WGU's patient base is at higher risk because it is so much more unhealthy than national averages. Ideally, I would create KPIs for all of the conditions I brought over from the NHANES dataset, but only two KPIs were required for this assessment, and I also ran out of room on the dashboard. To analyze further would require multiple dashboards connected in the form of a Tableau story.

Age is also used here as a benchmarch, though I am comparing the age distributions rather than rates. Without the external dataset, I would not know that the large number of people in the 80+ bin is rather abnormal compared to the nation, which signals that WGU has a lot more older patients than the average. The remaining condition variables are also interesting to look at in the context of the histogram. Conditions that are more frequently seen in older populations in the NHANES data are widespread in the WGU data, with younger people getting them at apparently similar rates to older people. Without the NHANES data, I might not see this as strange.

In conclusion, the external dataset allows for the benchmarking of WGU data against national data by allowing the comparison to national averages of the distribution of disease by age, the spread of disease by age, the rate diseases are diagnosed, and the distribution and spread of the WGU patient base as described in the examples above. The external data simply allows me to tell if WGU's patients have condition rates that are better, worse, or the same as national averages, and whether or not those conditions affect the same age and gender groups as in the national data.

## C3. Explanation of Data Representations
The histogram showing number of patients by age bucket is best utilized for decision making when one sets any one or multiple of the filters at the top of the dashboard. Slicing and dicing by condition in this way allows one to see the distribution of people with the condition(s) chosen. With WGU and CDC data side-by-side, a quick visual comparison can be made between the two. For example, if you set the arthritis filter to "yes," the histograms will update showing all patients with arthritis. One thing you will notice after doing this is that this disease is distributed differently for CDC patients than it is for WGU patients. On the CDC histogram, there is a peak around 60 years old, and the distribution is left-skewed, indicating that arthritis is more common in older patients for the CDC. On the WGU histogram, the distribution is more or less uniform, with an odd peak in the last, oldest age bucket (though this appears for the CDC too, and is due to the top-coding of ages higher than 80.) This means that arthritis is just as common amongst the youngest of the WGU patient base as it is for the older patients. The histograms work hand-in hand with the boxplots for decision making purposes.

The boxplots also delineate between CDC and WGU data for comparison. If we filter for arthritis equals "yes" like we did in the above scenario, the box plots will provide the mean and median age for each group, further validating our findings that WGU's spread of those with arthritis is uncommonly large compared to national data. The CDC data's mean and median are much higher and its spread much smaller. Besides looking at the mean and median lines, executives can see the CDC's spread is visually smaller by inspecting the length of the orange rectangle on each boxplot.

Using these graphs in tandem with each other along with the filters at the top of the dashboard, executives can see how WGU compares to the national rates of disease. Some diseases are risk-factors for readmission, which is the problem the hospital system is facing. Executives can use these visualizations together to figure out what groups of people need to be focused on (for example, young people with arthritis, which is abnormal compared to national rates) and identify which groups of people might benefit from programs that prevent preventable diseases or the committing of resources to screen for less preventable diseases to ensure patients get care as early as possible, before it transforms into a hospital stay, or worse yet, a readmission.

## C4. Explanation of Interactive Controls
I have provided many interactive controls for slicing the data, but here, I will only discuss two: the arthritis filter, which possesses the same design as the other condition filters, and the gender filter. One can use either by itself or both in combination.

The arthritis filter (and its other condition filter counterparts) are single dropdown selection filters. You have the option of choosing "Yes," indicating that you want to see data on patients *with* arthritis, "No," indicating that you want to see data on patients *without* arthritis, or the default, "(All)," which will show you patient data regardless of whether or not they have arthritis. Using the arthritis filter allows you to view a subset of the data in the dashboard. While the "(All)" selection provides you a broad, big picture view, choosing "Yes" or "No" allows you to gain more insight into the distribution shapes and spreads of subsets with or without arthritis versus the distribution shape and spread of the dataset as a whole.

The gender filter, meanwhile, is a multi-selection checkbox filter. This enables the user to select all the possible genders, or just one or two of them at a time. Like the above filter, making a selection in this filter provides a view into a subset of the data. For example, if you select "Female" in this filter, the dashboard's visuals will change to show just data on female patients." The user can also choose to select two choices, such as "non-binary" and "female" at the same time. The visuals on the dashboard will update to show data on the subset of patients who are either female or non-binary. Similar to the previous filter discussed, the "(All)" choice shows the entire dataset, regardless of gender. The user may also slice the dashboard's data by using the gender pie charts at the bottom. Clicking on slice of the pie will filter the dashboard by that pie slice's source (CDC or WGU) and gender. To be useful, it would be better that the user hold Ctrl on their keyboard and click two slices at a time, one from each source-- for example, the CDC's female slice and WGU's female slice. Failing to do so will result in one source's graphs disappearing from the dashboard altogether, since a source filter is included when a pie slice is selected.

## C5. Accessibility
The vast majority of people are red-green colorblind, meaning they have trouble distinguishing these two colors from each other. Less commonly, people may also be blue-yellow colorblind, which can take a couple of forms: tritanomaly and tritanopia. Tritanomaly makes distinguishing blue and green from each other difficult, as well as yellow and red. Tritanopia makes it difficult to tell the difference between blue and green, purple and red, and yellow and pink (National Eye Institute, 2023). Thus, to make this dashboard as colorblind-friendly to as many types of colorblindness as possible, I avoided the following color combinations when making the boxplots since a built in colorblind-friendly Tableau palette was not an option.

* Red and green
* Blue and green
* Yellow and red
* Purple and red
* Yellow and pink

For the pie charts, Tableau conveniently has a colorblind-friendly palette, which I used for these visuals. This palette is called "Color Blind." In Jeffrey Shaffer's article, there are visuals that show that blue and orange, for both deuteranopes and protanopes, are well-distinguished (Shaffer, 2016). In both cases, blue looks blue, and orange looks brownish-yellow.

However, just to be positive that the dashboard is colorblind-friendly, it is helpful to see what it would look like to people with various kinds of colorblindness with your own eyes-- so this is exactly what I did. In Shaffer's article, he recommends using a vision simulator to see what someone with each kind of vision might see: https://asada.website/webCVS/. Below, you will find pictures of my results.

### Deuteranopia
![Deuteranopia](https://i.imgur.com/C5PPV5Y.png)

### Protanopia
![Protanopia](https://i.imgur.com/cDpa950.png)

### Tritanopia
![Tritanopia](https://i.imgur.com/0Sv7mPP.png)

In the two red-green forms of colorblindness, deuteranopia and protanopia the dashboard looks almost true to the original colors, with the orange just a bit duller. The colors are easily distinguishable. For the blue-yellow colorblindness, tritanopia, the colors are altogether different. Instead of blue, gray, and orange, they appear teal, gray, and pink, but they are still visually pleasing and easily distinguishable as well.

For those with complete colorblindness (those who see no color at all,) hovering over the pie chart sections produces a tooltip that helpfully identifies which gender each pie chart section belongs to. Even in shades of black, white and gray, the boxplots should be readable, since color is more of a stylistic choice on this visual-- color provides no data to be gleaned on a boxplot. 

While I have no screenshot for complete colorblindness, I did test this similarly to how I tested deuteranopia, protanopia, and tritanopia. I found a website that converted an image of my dashboard to grayscale, and in doing so, I was able to understand how a person with complete colorblindness might view the dashboard. The gray and orange are able to be distinguished, though it is much harder than with color vision of any kind. Blue looks like a very dark gray, while orange appears to be a much lighter gray, so the boxplots remain readable since the shades of gray are so different that they do not obscure each other.

## C6. Supporting the Data Story
One of the main points I wanted to make in my data story was how oddly uniform WGU's data is compared to the nation as a whole-- no matter the subset you're viewing. The histograms and the bar charts both illustrate this nicely. Conditions that are typically thought of as affecting the older population show as expected in the CDC-sourced histogram, but the WGU histogram shows roughly the same number of people having the condition no matter their age. For example, you can see the typical left-skewed histogram on the CDC histogram if you set the arthritis filter to "yes," while the WGU histogram is flat, except for the final bucket, which appears as a spike due to top-coding those older than 80. The boxplots only serve to drive home this pattern-- the spread of those with arthritis for the CDC is much smaller and on the higher end of the age spectrum, with the mean and median much higher than WGU's boxplot, where the spread is much wider. Because of the uniformity of WGU's patient base, groups one might not expect to be at risk for age-related diseases appear to be at higher risk than the general population, which could lead to their being overlooked by physicians, and thus, increase their chances of readmission. Reducing readmission being the goal of my story, this is important to visualize.

Even the bar charts show this pattern. The average number of children is a right-skewed shape in the bar chart for the CDC. Younger people have more children at home than older people. But for the WGU data, the bar chart is very uniform. Even the oldest people in the WGU dataset, 80 year olds and older, have about two minors at home in their household.

The histogram also indicates that there are far more people 80 and over in the WGU patient base than there are in the general population surveyed by the CDC, which is helpful to note. Since the purpose of my story is to identify ways to reduce readmission risk factors, understanding that WGU's patient base contains quite a lot of older people, which is what I would consider one of these risk factors, is important to my story.

## C7. Audience Analysis
The WGU medical data dictionary indicates that the audience for this dashboard presentation is a group of executives. Executives are generally more interested in the big picture rather than the minor details, though the ability to slice down a level or two can be useful to them, so long as the slicing is quick and simple. There are five steps to audience analysis in data storytelling: identify, understand their background, determine interest, recognize preconceptions, and assess data literacy (Chip, 2023). I'll go through these one by one.

### Identify the Audience
I gleaned this information from the data dictionary, which introduces each of the audience members to whom I'll be speaking. 

* Senior Vice President of Hospital Operations (SVP)
* Vice President of Research (VP)
* Panel of Regional Vice Presidents (Regional VPs)

I know that they are each executive leaders. Since the audience is composed of decision-makers, I focused on talking more about the big picture implications rather than getting into the minute details of the data. In a similar vein, I will avoid using statistical concepts that they may not have the knowledge to comprehend and will instead stick to using commonly used figures like averages when speaking to them (Mattison, 2023). Lastly, I made the dashboard simple to filter so that finding new insights is quick and easy for them, as executives are often on the go and value speed.

### Understand the Audience's Background
First, let's understand the amount of education each audience member is likely to have, and in what fields.

The Senior VP of Hospital Operations is likely to have a bachelor's degree at the least, and usually in operations management or business administration. Often, a person in this position would also have quite a lot of experience in working in operations (Powers, 2023). Because the medical dictionary states that this person oversees operations and often looks at trends, it is evident to me that this audience member is not well versed in data science concepts. This person is likely well versed in trend analysis, though. If I've learned anything as a business analyst, this usually means that they are excellent at analyzing a graph for patterns. Knowing this, I adapted my message so that I refrain from using data science concepts this executive, and the others below, may not understand.

Likewise, the VP of Research is probably lacking in the same way. While this person likely has degrees outside of data science, they will not be well versed in data science concepts. Thus, my message will be amended in the same way I wid for the Senior VP of Hospital Operations: I will avoid using unruly data science concepts this person will not understand and keep it to simpler, descriptive statistics they are likely to have used before, such as averages, medians, and the shape and spread of graphs.

Finally, we have a panel of many Regional Vice Presidents. These people are likely to have a similar education to the Senior VP of Hospital Operations, as they too are responsible for managing operations along with the Senior VP. However, they may have less experience in operations. Likewise, my message is adapted the same way it was for the Senior VP.

### Determine Audience Interest
Each person in attendance to this presentation has their own interest in doing so, usually pertaining to their position in the company and what business functions they oversee.

The Senior VP of Hospital Operations oversees all locations, comes up with and creates new programs that lead to healthier patients based on trend analysis, and is interested in regional demographics. Because this person's job is so broad, my message will be tailored to the big picture, as I've stated several times now. Since they are interested in demographics, I was careful to include visuals on such things that are easy to slice that I can speak to when telling my data story. This person's interest in trends also lends itself to the external dataset I chose, as in my message, I will be sure to focus on benchmarking the WGU hospital system against national averages, especially in regard to the issue discussed earlier-- the odd uniformity of WGU's data. With this kind of information, this person would be able to identify groups for which to develop new initiatives that might reduce the risk factors for readmission-- and improve patient outcomes. Such is this person's job description.

The VP of Research oversees new research and identifies patterns that they might use to improve patient care through strategy. This person's job is also very broad, though they seem to be more technical than the Senior VP of Hospital Operations. This person would benefit from much the same as I described above. Patterns are more specific than larger, overall trends, and this person studies patterns. My message will be tailored to uncover patterns this person might find useful, such as how the WGU hospital system's data is very uniform no matter the filter applied, meaning there are likely groups of people for whom one might assume there is no risk for diseases that typically only afflict the older population. The VP of Research may be able to use this information to drive better patient outcomes by creating strategic programs to target these groups.

Lastly, the panel of Regional VPs have very similar duties to the Senior VP, though they are likely to be more interested in data that applies to their specific region. While this presentation does not delve that deep, it is still beneficial for them to understand the hospital system's trends and patterns as a whole so they understand why new programs and strategies are being developed and deployed. Unfortunately, due to the lack of available data, this audience group will likely wish for more granular information I simply do not have.

Executives like these are typically looking for the broad strokes-- information that they can use to make high-level decisions and create sweeping programs and initiatives. Thus, my message will focus on that. As an analyst, I will be making recommendations at the conclusion of the presentation, supported by evidence from the visualizations, that these executives can use to perform the duties required of them, as detailed above.

### Recognize the Audience's Preconceptions
This step is difficult to perform having never met these hypothetical hospital executives. However, having worked as a business analyst for four years, it is my understanding that executives often view their own company through rose-colored glasses. However, these executives are well-versed in strategic planning and trend analysis, so they are likely to be more logical. They will jump on patterns and trends and seek to brainstorm the fix. Thus, it is important to discuss each pattern or trend revealed in the presentation in a clear and timely manner and follow the presentation with recommendations for action.

### Assess Data Literacy
I have already spoken to this in the above steps. A group of executives is unlikely to be data literate in the same way an analyst is. Thus, my message is adapted to avoid getting to mired in granularity and complex statistical concepts. In my four years as a business analyst, every business-minded, business-educated executive I've worked with has had a basic understanding of descriptive statistics, distribution shape and spread, but nothing more than that. My presentation, therefore, keeps the statistical concepts to the minimum, using only average, median, quartiles, and basic distribution shapes to describe the data and the patterns therein.

## C8. Universal Access
I have designed my presentation for universal access by ensuring the dashboard itself has carefully designed layouts for both desktop and phone. While creating these layouts, I made sure they were easy to use on every phone model preview available in Tableau. I did the same for every desktop monitor size. Thus, the presentation is optimized for most devices that could be used to access it. With the mobile layout, access can even be had on the go.

My dashboard is available on Tableau Public, meaning one only needs a free account to be able to view it. There is no requirement to download software, nor a paywall.

For those that are blind or visually impaired, the Panopto video provides a means of consuming the dashboard via a purely auditory route. For those who are deaf or hard of hearing, I have created a Google Slides presentation to go along with the dashboard. These slides contain my introduction, the names of the data sources and the kind of data they contain, the key takeaways from the narrative presentation, and my recommendations in a readable format. By visually exploring the easy-to-use dashboard, they can strengthen their understanding of the slides. The link to the slides is provided in the description of the Tableau Public dashboard, so it is easy to find.

## C9. Effective Storytelling Elements
According to Harvard Business School, there are three elements of data storytelling: data, narrative, and visualizations (Data Storytelling: How to Tell a Story With Data, 2021). I will discuss two here.

### Narrative
The most important element, in my opinion, is the narrative. I supported my narrative by first introducing myself and providing my background. This was to lend myself credibility as a presenter. Afterall, if the presenter isn't credible, why would one listen? In my narrative, I also provided the context of the data: the goal of the analysis and the origin of the data sources and the type of data they contain. A good narrative needs a problem to solve or a goal to achieve. In this case, the "problem to solve" is a question. What can we identify as risk factors that could reduce readmission rates? After introducing myself and establishing context, I use my narrative to discuss the details of this question using *data and visualizations* and provide supporting evidence for the key takeaways I wanted to highlight:

* The WGU patient base distribution is very uniform, no matter the subset. As a result, there are groups of WGU patients that may be neglected because they are afflicted by diseases not common to their age group.
* The WGU patient base contains substantially more geriatric patients than average.
* WGU patients are diagnosed with high blood pressure and obesity at substantially higher rates than the national average.

These points are directly related to the goal of my narrative. In fact, they are the very risk factors we wanted to identify. Using these, I finish my narrative by providing recommendations for action based on the data-supported key takeaways.

Like the plot of a work of fiction, we have a storyline. In the beginning, I introduce the characters: myself, the presenter, and the two groups of people involved in answering this question, the CDC patient base and the WGU patient base. I provide context surrounding the data to set the scene. In the middle, I spend my time providing evidence for the conclusions I want to draw, often in the form of visualizations or KPIs. I use this evidence to support my claims-- the key takeaways-- and conclude my story by providing a call to action for the audience.

### Visualizations
I used several visuals in my storytelling: histograms, bar charts, pie charts, and boxplots. I aimed to keep these visuals simple to comprehend and easy to filter. These visualizations, as referenced above, helped to support my narrative by providing evidence for my claims as I spoke about them. Whilst telling my data story, I could, in real time, filter the data to show precisely the point I was trying to illustrate, adding to the credibility of the point I was trying to lead the audience to and keeping the audience engaged. In addition, had this presentation been in person or via a live webinar, I could respond to questions in real time, using the visualizations and filters to answer any questions that arose, which would serve to keep the audience engaged in the discussion.

## D. Sources

Blood Pressure & Cholesterol (BPQ_J). (2020, February). CDC. Retrieved April 19, 2024, from https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/BPQ_J.htm

Chip. (2023, November 30). Data storytelling: the art of knowing your audience. QuantHub.<br>
&emsp;&emsp; https://www.quanthub.com/data-storytelling-the-art-of-knowing-your-audience/#:~:text=Understanding%20your%20audience%20helps%20you,insights%20that%20resonate%20with%20them.

Data Storytelling: How to Tell a Story with Data. (2021, November 23). Business Insights Blog. https://online.hbs.edu/blog/post/data-storytelling

Demographic Variables and Sample Weights (DEMO_J). (2020, February). CDC. Retrieved April 19, 2024, from<br>
&emsp;&emsp; https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/DEMO_J.htm

Diabetes (DIQ_J). (2020, February). CDC. Retrieved April 19, 2024, from https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/DIQ_J.htm

Mattison, R. (2023, July 19). Data storytelling: How to tell a great story with data. ThoughtSpot.<br>
&emsp;&emsp; https://www.thoughtspot.com/data-trends/best-practices/data-storytelling

Medical Conditions (MCQ_J). (2020, February). CDC. Retrieved April 19, 2024, from https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/MCQ_J.htm

National Eye Institute. (2023, August 7). Types of Color Vision Deficiency | National Eye Institute. Retrieved April 22, 2024, from<br>
&emsp;&emsp; https://www.nei.nih.gov/learn-about-eye-health/eye-conditions-and-diseases/color-blindness/types-color-vision-deficiency

Nelson, M. (2023, August 8). *D206: Data Cleaning Performance Assessment.* Unpublished manuscript, Western Governors University.

NHANES questionnaires, datasets, and related documentation. (n.d.). CDC. Retrieved April 19, 2024, from<br>
&emsp;&emsp; https://wwwn.cdc.gov/nchs/nhanes/continuousnhanes/default.aspx?BeginYear=2017

Powers, J. (2023, January 27). What is a Vice President of Operations? how to become one, salary, skills. Built In.<br>
&emsp;&emsp; https://builtin.com/learn/careers/vice-president-operations

Shaffer, J. (2016, April 20). 5 Tips on Designing Colorblind-Friendly Visualizations. Tableau. Retrieved April 22, 2024, from<br>
&emsp;&emsp; https://www.tableau.com/blog/examining-data-viz-rules-dont-use-red-green-together